# Gaia data visualization using voila and vaex
This notebook shows how to create a fully interactive dashboard with a Sky plot and a Hertzsprung-Russell (HR) diagram, which [can be rendered as dashboard using Voila](https://voila.vaex.io/voila/render/gde-examples/30-voila-vaex-sky-hr-diagram.ipynb).

For this, we use the following Data/Software:

 * <a href="https://gea.esac.esa.int/archive/">Gaia</a> EDR3 dataset, crossmatched to the <a href="https://panstarrs.stsci.edu/">Pan-STARRS</a> dataset (crossmatched done by Sergey Kopsov).
 * [Voila](https://voila.readthedocs.io/) to render as dashboard.
 * [Vaex](https://github.com/vaexio/vaex) to do the data aggregation.
 * [Vaex-jupyter](https://vaex.io/docs/tutorial_jupyter.html) to make the high level interactive widgets.
 * [ipyvuetify](https://github.com/mariobuikhuizen/ipyvuetify) which is what vaex-jupyter builds on top of, and wraps [vuetifyJS](http://vuetifyjs.com/) material design widgets for a modern look.
 * [voila-vuetify](https://github.com/voila-dashboards/voila-vuetify/) The voila template that renders the notebook without output cells, but only renders widgets with a `mount_id`.

In [ ]:
import vaex
import vaex.jupyter
import numpy as np

In [ ]:
df = vaex.open('ws://dataframe.vaex.io/gaia-edr3-x-ps1')#[:10_000_000]

In [ ]:
# main sky plot parameters
limits_l = [360, 0]
limits_b = [-90, 90]
limits_g = [12, 22]
shape_g = 100

# main cmd parameters
limits_color = [-0.5, 1.5]
limits_magnitude = [22, -10]

# adding in virtual columns
color = df.phot_bp_mean_mag - df.phot_g_mean_mag
df['distance'] = 1/df.parallax
df['M_g'] = df.phot_g_mean_mag - 5 * np.log10(df.distance) - 10
magnitude = df.M_g
limits_color = [-0.5, 1.5]
limits_magnitude = [22, -10]
df['magnitude'] = magnitude
df['color'] = color

extra_filter = "distance < 0.1"
df.select(extra_filter, name="distance_filter")
df.select('True', name="sky_filter")



In [ ]:
widget_sky = df.widget.heatmap(df.l, df.b, limits=[limits_l, limits_b], selections=['distance_filter', 'distance_filter & sky_filter'], selection_interact='sky_filter')
widget_sky

In [ ]:
widget_selection_expression = df.widget.selection_expression(extra_filter, name="distance_filter")
widget_selection_expression

In [ ]:
widget_cmd = df.widget.heatmap(df.color, df.magnitude,
                                limits=[limits_color, limits_magnitude],
                                selections=['distance_filter & sky_filter'])
widget_cmd

In [ ]:
df.count(selection='sky_filter')

In [ ]:
df.selection_histories['sky_filter'][0].boolean_expression

In [ ]:
card1 = vaex.jupyter.widgets.ContainerCard(title='Sky plot',
              main=widget_sky,
              controls=[widget_selection_expression],
              show_controls=True,
              card_props={'style': 'min-width: 350px;'})
card1

In [ ]:
card2 = vaex.jupyter.widgets.ContainerCard(title='HR Diagram',
              main=widget_cmd,
              controls=[],
              show_controls=False,
              card_props={'style': 'min-width: 350px;'})
card2

In [ ]:
# all cards are 12 columns wide on small screen, 6 on medium, 4 or large
# see https://vuetifyjs.com/en/components/grids/#grid-system
import ipyvuetify as v
v.Container(fluid=True, _metadata={'mount_id': 'content-main'}, children=[
    v.Row(children=[
        v.Col(lg=4, md=6, sm=12, children=[
            card1
        ]),
        v.Col(lg=4, md=6, sm=12, children=[
            card2
        ]),
    ])
])